In [76]:
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
import numpy as np
import os
import osmnx as ox
import rasterio
from rasterstats import zonal_stats
from rasterio.plot import show
from rasterio.enums import Resampling
import geopandas as gpd
from shapely.geometry import Polygon
import json
import math

In [77]:
# CONSTANTS

# Base Coordinate System EPSG code 
BASE_CRS = 4326 

# Location center [minx, miny, maxx, maxy]
# LOCS = [[-19.3758794, 47.33124417, -19.07561176, 47.63146114]]
LOCS = [[47.34663,-19.09486, 47.60648,-19.35696]]
# LOCS = [[-96, 31, -82, 39], [55, 2.38, 101, 38]]
# LOCS = [[42.71,-9.28, 60.71,-27.66]]

# Raster directory
DIR = 'layers/'

# Which sampling statistic to output
STATS=['mean']

In [86]:
output = {}
output["Locations"] = LOCS

for ind, LOC in enumerate(LOCS):

    # Initialize empty holders
    tifs = []
    shps = []
    loc_output = {}

    # Add constants to location output file
    loc_output["Location"] = LOC
    loc_output["Stats"] = STATS
    loc_output["Bands"] = {}

    # Iterate through directory
    for subdir, dirs, files in os.walk(DIR):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif"):
                tifs.append(path)
            elif path.endswith(".shp"):
                shps.append(path)

    for ds_path in tifs:
        # Select raster image 
        print("\nOpening " + ds_path)
        ds = gdal.Open(ds_path)

        # Get the number of layers
        print("Number of Layers: " + str(ds.RasterCount))
#         print(dir(ds.GetRasterBand(0).GetStatistics(True, True))

        # Get source coordinate reference type 
        projection = int(osr.SpatialReference(wkt=ds.GetProjection()).GetAttrValue('AUTHORITY',1))
        print("EPSG: " + str(projection))
        source = osr.SpatialReference()
        source.ImportFromEPSG(projection)

        # Set target coordinate reference 
        target = osr.SpatialReference()
        target.ImportFromEPSG(BASE_CRS)

        # Create point transformation
        transform = osr.CoordinateTransformation(source, target)

        # Get raster coordinates (rc) in source reference 
        rc = getRasterCoords(ds)
#         print("Raster coordinates in source Ref: " + str(rc))

        # Convert raster coordinates to target reference
        rc_conv = np.array([])
        y,x,z = transform.TransformPoint(rc[0], rc[1])
        rc_conv = np.concatenate((rc_conv, [x, y]))
        y,x,z = transform.TransformPoint(rc[2], rc[3])
        rc_conv = np.concatenate((rc_conv, [x, y]))
#         print("Raster coordinates in target reference: " + str(rc_conv))

        # Check if raster contains the desired location
        if rasterContainsLoc(rc_conv):

            print("Processing file " + os.path.basename(ds_path) + "...")
            for band in range(1, ds.RasterCount+1):
                model = rasterio.open(ds_path)
                crs = {'init': 'epsg:'+str(BASE_CRS)}
                polygon = gpd.GeoDataFrame(crs=crs, geometry=[returnPolygon(LOC)])
                place = polygon.to_crs(crs=model.crs)

                # Optional - Use location name as Polygon
#                 place = ox.geocode_to_gdf("California")
#                 place = ox.geocode_to_gdf(polygon)
#                 place = place.to_crs(crs=model.crs)

                # Optional - Visualize band 
#                 ax = place.plot(facecolor='None', edgecolor='red', linewidth=2)
#                 show((model, band), ax=ax)

                # Compute zonal stats 
                array = model.read(band)
                affine = model.transform

                if returnPolygon(LOC).area < getPixelSize(ds)[0]*getPixelSize(ds)[1]:
                    # Check if resample is needed
#                     print("Resampling image from area " + str(getPixelSize(ds)[0]))
                    upscale_factor = getPixelSize(ds)[0]*getPixelSize(ds)[1] / returnPolygon(LOC).area
                    array = model.read(
                        band,
                        out_shape=(
                            int(model.height * upscale_factor),
                            int(model.width * upscale_factor)
                        ),
                        resampling=Resampling.bilinear)
                    affine = model.transform * model.transform.scale(
                        (model.width / array.shape[-1]),
                        (model.height / array.shape[-2])
                    )
                
                zs = zonal_stats(place, array, affine=affine) 
                    
                # Write results to output dict
                loc_output["Bands"][os.path.basename(ds_path) + "-" + str(band)] = zs[0]

            # Finished processing file bands
            print("Complete")

        # If raster does not contain desired location
        else:
            print("ERROR: Raster does not contain location: " + str(LOC) + ". Will not be included in results output.")

    output[ind] = loc_output
    
print("\nWriting location results to results.json...")
with open('result.json', 'w') as fp:
    json.dump(output, fp, indent=2)
print("Complete")



Opening layers//img.tif
Number of Layers: 20
EPSG: 3857
Processing file img.tif...
Complete

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Processing file basic.tif...
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.8983152841195214
Resampling image from area 0.89831528

In [82]:
def getPixelSize(ds):
    
    gt = ds.GetGeoTransform()
    return(transform.TransformPoint(gt[1], -gt[5]))

In [83]:
def getRasterCoords(ds):
    
    gt = ds.GetGeoTransform()
    w = ds.RasterXSize
    h = ds.RasterYSize
    
    minx = gt[0]
    maxy = gt[3] 
    miny = gt[3] + w*gt[4] + h*gt[5] 
    maxx = gt[0] + w*gt[1] + h*gt[2]
    
    return[minx, miny, maxx, maxy]

In [84]:
def rasterContainsLoc(rc):
    if LOC[0] >= rc[0] and LOC[3] <= rc[3] and LOC[1] >= rc[1] and LOC[3] <= rc[3]:
        return True
    else:
        return False

In [85]:
def returnPolygon(bounds):
    poly = Polygon([(LOC[0], LOC[1]), (LOC[2], LOC[1]), (LOC[2], LOC[3]), (LOC[0], LOC[3]), (LOC[0], LOC[1])])
    return(poly)